# Process Mining Functions

In [4]:
import os
import pandas as pd
import numpy as np
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.petrinet import visualizer as pn_visualizater
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualizer
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.filtering.log.paths import paths_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.util import constants
# import sqlalchemy
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
root_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\\ProcessMiningData'
raw_data = pd.read_csv(os.path.join(root_path, "x.csv"), names=['target', 'id', 'date', 'flag', 'user', 'text'], header=None)

In [ ]:
df['date'] = pd.to_datetime(df.date)
df = df.sort_values(by=[''])
df.rename(columns = {'CASE_ID': 'case:clientID', 'ACTIVIT': 'concept:name', 'ACTIVITY_DATE': 'time:timestamp'}, inplace = True)
parameters = {
    log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'case:clientID'
}

In [ ]:
def view_process(log, file):
    #create and visualize the process map
    filtered_log = log
    heu_net = heuristics_miner.apply_heu(filtered_log,
                                        parameters = {heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_DFG_OCCURRENCES})
    gviz = hn_visualizer.apply(heu_net)
    print()
    print('Frequency')
    
    hn_visualizer.view(gviz)
    
    workflow_name = root_path + '/' + file + '_workflow.png'
    
    print(workflow_name)
    hn_visualizer.save(gviz, workflow_name)
    
    #create and visualize the process performance chart
    dfg = dfg_discovery.apply(filtered_log, variant = dfg_discovery.Variants.PERFORMANCE)
    parameters = {dfg_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
    gviz = dfg_visualizer.apply(dfg, log = filtered_log, variant = dfg_visualizer.Variants.PERFORMANCE, parameters)
    
    print()
    print('timing')
    print()
    
    dfg_visulizer.view(gviz)
    
    print()
    variants_count = case_statistics.get_variant_statistics(filtered_log)
    variants_count = sorted(variants_count, key = lambda x: x['count'], reverse = True)
    print()
    print('top path timing')
    print()
    print('policy counts:')
    print((variants_count[0]['count']))
    print()
    
    filtered_log1 = variants_filter.apply(filtered_log, [variants_count[0]['variant']])
    dfg = dfg_discovery.apply(filtered_log1, variant = dfg_discovery.Variants.PERFORMANCE)
    parameters = {dfg_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
    gviz = dfg_visualizer.apply(dfg, log = filtered_log1, variant = dfg_visualizer.Variants.PERFORMANCE, parameters)
    
    dfg_visulizer.view(gviz)

In [ ]:
df_non_nigo_hipaa = df[df['nigo_auto_hipaa_ind']==0]
log_non_nigo_hipaa = log_converter.apply(df_non_nigo_hipaa, parameters = parameters, variant = log_converter.Variants.TO_EVENT_LOG)
print(df_non_nigo_hipaa['case:clientID'].unique().shape)
viewrequirement(log_non_nigo_hipaa,'non_nigo_hipaa_cases')